# ID 3 - Stirbu Larisa Mihaela E1


## 1. Preprocesare

#### a) Descriere set de date
Setul de date cu pinguinii Palmer contine informatii despre trei specii de pinguini (Adelie, Chinstrap, Gentoo) colectate in timpul unui studiu in Insula Palmer, Antarctica.


Setul de date contine urmatoarele atribute pentru fiecare pinguin:

**species**: Specia pinguinului (Adelie, Chinstrap, Gentoo)

**island**: Insula pe care a fost gasit pinguinul (Biscoe, Dream, Torgersen)

**bill_length_mm**: Lungimea ciocului in mm

**bill_depth_mm**: Adancimea ciocului în mm

**flipper_length_mm**: Lungimea aripioarei in mm

**body_mass_g**: Greutatea pinguinului in grame

**sex**: Genul pinguinului (femela, mascul)

#### b) Eliminarea liniilor incomplete

In [1]:
import json
import math
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
import matplotlib.pyplot as plt

In [2]:
file = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv'

continuous_attributes = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
discrete_attributes = ['island', 'sex']
all_attributes = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'island', 'sex']
target_attribute = "species"

def process_data(file):

    data = pd.read_csv(file)
    data.dropna(inplace=True)   ## liniile incomplete sunt eliminate
    print(data)
    return data

data = process_data(file)
print(data)

    species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0    Adelie  Torgersen            39.1           18.7              181.0   
1    Adelie  Torgersen            39.5           17.4              186.0   
2    Adelie  Torgersen            40.3           18.0              195.0   
4    Adelie  Torgersen            36.7           19.3              193.0   
5    Adelie  Torgersen            39.3           20.6              190.0   
..      ...        ...             ...            ...                ...   
338  Gentoo     Biscoe            47.2           13.7              214.0   
340  Gentoo     Biscoe            46.8           14.3              215.0   
341  Gentoo     Biscoe            50.4           15.7              222.0   
342  Gentoo     Biscoe            45.2           14.8              212.0   
343  Gentoo     Biscoe            49.9           16.1              213.0   

     body_mass_g     sex  
0         3750.0    MALE  
1         3800.0  FEMALE  
2     

#### c) Calculul mediei si a variantei

In [3]:
def compute_mean_and_variance(data, discrete_attributes):
    mean_and_variance = {}

    for attribute_name in discrete_attributes:
        if attribute_name in data.columns:
            attribute_values = data[attribute_name]

            if pd.api.types.is_numeric_dtype(attribute_values):
                mean = attribute_values.astype(float).mean()
                variance = attribute_values.astype(float).var()
            else:
                print(f"{attribute_name} non numeric attribute")
                mean = None
                variance = None

            mean_and_variance[attribute_name] = (mean, variance)

    return mean_and_variance

mean_and_variance = compute_mean_and_variance(data,discrete_attributes)
print(mean_and_variance)

island non numeric attribute
sex non numeric attribute
{'island': (None, None), 'sex': (None, None)}


## 2. Probabilitati, Teoria Informatiei

#### a) calculul probabilitatilor

In [4]:
def compute_probabilities(data, attribute):

    pmf = {}  

    attribute_values = data[attribute]
    total_samples = len(attribute_values)

    unique_values = attribute_values.unique()

    for value in unique_values:
        count = attribute_values.value_counts()[value]
        probability = count / total_samples
        pmf[value] = probability

    return pmf

pmf = compute_probabilities(data, "island")
print(pmf)

{'Torgersen': 0.14114114114114115, 'Biscoe': 0.4894894894894895, 'Dream': 0.36936936936936937}


#### a') calculul probabilitatilor conditionale
voi avea nevoie mai tarziu

In [5]:
def compute_conditional_probability(data, X, Y, value_X, value_Y):

    ###P(Y|X)=P(Y,X)/P(X) in the dataset.

    count_XY = len(data[(data[X] == value_X) & (data[Y] == value_Y)])
    count_X = len(data[data[X] == value_X])

    if count_X > 0:
        conditional_probability = count_XY / count_X
    else:
        conditional_probability = 0

    return conditional_probability 

print (compute_conditional_probability(data,"island", target_attribute, "Biscoe","Gentoo" ))

0.7300613496932515


#### b) calculul entropiei variabilelor discrete

In [6]:
def calculate_entropy(probabilities):

    entropy_value = -sum(p * math.log2(p) for p in probabilities.values() if p > 0)
    return entropy_value

pmf = compute_probabilities(data, "island")
print(calculate_entropy(pmf))

1.4339199606891568


#### c) calculul entropiei conditionale medii (cu atribute discrete)

In [7]:
def calculate_conditional_entropy(data, discrete_attribute, target_attribute):

    conditional_entropy = 0
    prob_discrete_attribute = compute_probabilities(data, discrete_attribute)  ## dictionar
    discrete_atr_val = prob_discrete_attribute.keys()  ##list val
    target_atr_val = data[target_attribute].unique()  ## list val


    for x in discrete_atr_val:  ##valorile atrb discret
        for y in target_atr_val:  ##val atrb target

            prob_target_discrete = compute_conditional_probability(data,discrete_attribute,target_attribute,x,y)
            if(prob_target_discrete != 0):
                conditional_entropy = conditional_entropy + prob_discrete_attribute.get(x)  * ( prob_target_discrete * math.log2(prob_target_discrete) )

    conditional_entropy = -conditional_entropy

    return conditional_entropy

print(calculate_conditional_entropy(data,"island", "species"))

0.7782324648463256


#### d) calculul castigului de informatie

In [8]:
def calculate_information_gain(data, target_attribute, other_attribute):
    ## H(Y)
    target_prob = compute_probabilities(data,target_attribute)
    target_entropy = calculate_entropy(target_prob)

    ## H(Y|X)
    target_other_entropy = calculate_conditional_entropy(data,other_attribute,target_attribute)

    info_gain_target_other = target_entropy - target_other_entropy
    return info_gain_target_other

print(calculate_information_gain(data, target_attribute,"island"))

0.7418510895067659


## 3. ID3

#### a) functia care gaseste atributul utilizat ca radacina a arborelui de decizie

In [9]:
def find_root_node(data, target_attribute, attributes):

    max_info_gain = float('-inf')
    best_attribute = None

    for attribute in attributes:
        info_gain = calculate_information_gain(data, target_attribute, attribute)

        if info_gain > max_info_gain:
            max_info_gain = info_gain
            best_attribute = attribute

    return best_attribute, max_info_gain
print(find_root_node(data, target_attribute, discrete_attributes))

('island', 0.7418510895067659)


atributul este : **island** (cand se calculeaza excluzand atributele continue)

#### b) implementarea functiei ID3 pe atribute discrete

In [10]:
def ID3_discrete(data, target_attribute, attributes, parent_node_class=None): 
    ##parent node retine valoarea nodului parinte care a fost aleasa pentru a ajunge in subarborele curent

    # se returneaza o frunza daca nu mai sunt disponibile alte accesari
    unique_values_count = len(data[target_attribute].unique())
    if len(data[target_attribute].unique()) == 1:
        leaf_value = data[target_attribute].iloc[0]
        return {target_attribute: leaf_value}

    # returneaza nod frunza daca nu mai sunt atribute, si se alege majoritar daca entropia nu e 0
    if len(data.columns) == 1:
        majority_class = data[target_attribute].mode().iloc[0]  ## mode = frecventa
        return {target_attribute: majority_class}

    # gaseste radacina pt subarbore
    best_attribute, max_info_gain = find_root_node(data, target_attribute, attributes)

    if best_attribute is None:
        majority_class = data[target_attribute].mode().iloc[0]
        return {target_attribute: majority_class}

    # crearea arborelui
    tree_node = {
        "node_attribute": best_attribute,
        "n_observations": data[best_attribute].value_counts().to_dict(),
        "information_gain": max_info_gain,
        "values": {}
    }

    attributes = [attr for attr in attributes if attr != best_attribute]

    # crearea recursiva a subarborilor
    for value in data[best_attribute].unique():
        subset_data = data[data[best_attribute] == value].drop(columns=[best_attribute])
        subtree = ID3_discrete(subset_data, target_attribute,attributes, parent_node_class=value)
        # se adauga subarborele
        tree_node["values"][value] = subtree

    return tree_node

mytree = ID3_discrete(data,"species",discrete_attributes)
print(json.dumps(mytree, indent=4))

{
    "node_attribute": "island",
    "n_observations": {
        "Biscoe": 163,
        "Dream": 123,
        "Torgersen": 47
    },
    "information_gain": 0.7418510895067659,
    "values": {
        "Torgersen": {
            "species": "Adelie"
        },
        "Biscoe": {
            "node_attribute": "sex",
            "n_observations": {
                "MALE": 83,
                "FEMALE": 80
            },
            "information_gain": 9.036945995000156e-05,
            "values": {
                "FEMALE": {
                    "species": "Gentoo"
                },
                "MALE": {
                    "species": "Gentoo"
                }
            }
        },
        "Dream": {
            "node_attribute": "sex",
            "n_observations": {
                "MALE": 62,
                "FEMALE": 61
            },
            "information_gain": 5.895490710949858e-05,
            "values": {
                "FEMALE": {
                    "species": "Chins

#### c) compararea rezultatelor implementarii proprii vs sklearn

Algoritmul implementat de mine permite si utilizarea variabilelor care nu sunt numerice, sau permit mai mult de 2 valori (spre exemplu island). Inaltimea arborelui de decizie este 3.
Algoritmul implementat cu libraria sklearn trebuie sa "converteasca" atributele non-numerice in atribute numerice, acesta realizand compasi de decizie binari de tipul (daca ... <0.5 : true else false). Inaltimea arborelui de decizie este 4.
Cu cat sunt mai multe valori posibile pentru fiecare atribut, inaltimea arborelui generat de ID3 cu sklearn poate creste.(in cazul variabilelor discrete, categorice)

#### c') functia ID3 pe atribute discrete implememntata cu **sklearn**

In [11]:

def ID3_discrete_sklearn(data):
    X = data.drop("species", axis=1)
    y = data["species"]

    # transformarea variabilelor "categorical" in variabile numerice
    X_encoded = pd.get_dummies(X)
    # arbore de decizie
    tree = DecisionTreeClassifier(criterion='entropy', random_state=42).fit(X_encoded,y)

    tree_rules = export_text(tree, feature_names=list(X_encoded.columns))
    print(tree_rules)
    
ID3_discrete_sklearn(data)

|--- flipper_length_mm <= 206.50
|   |--- bill_length_mm <= 43.35
|   |   |--- bill_length_mm <= 42.35
|   |   |   |--- bill_depth_mm <= 16.65
|   |   |   |   |--- bill_length_mm <= 39.50
|   |   |   |   |   |--- class: Adelie
|   |   |   |   |--- bill_length_mm >  39.50
|   |   |   |   |   |--- class: Chinstrap
|   |   |   |--- bill_depth_mm >  16.65
|   |   |   |   |--- class: Adelie
|   |   |--- bill_length_mm >  42.35
|   |   |   |--- flipper_length_mm <= 189.50
|   |   |   |   |--- class: Chinstrap
|   |   |   |--- flipper_length_mm >  189.50
|   |   |   |   |--- class: Adelie
|   |--- bill_length_mm >  43.35
|   |   |--- island_Dream <= 0.50
|   |   |   |--- sex_FEMALE <= 0.50
|   |   |   |   |--- class: Adelie
|   |   |   |--- sex_FEMALE >  0.50
|   |   |   |   |--- class: Gentoo
|   |   |--- island_Dream >  0.50
|   |   |   |--- bill_length_mm <= 44.65
|   |   |   |   |--- bill_depth_mm <= 18.90
|   |   |   |   |   |--- class: Chinstrap
|   |   |   |   |--- bill_depth_mm >  18.

#### d) functia care calculeaza pragurile de separare pe atributele continue


In [12]:
def get_splits(continuous_attribute, labels):
    data = pd.DataFrame({'attribute': continuous_attribute, 'label': labels})
    data = data.sort_values(by='attribute')

    splits = []
    for i in range(1, len(data)):
        if data['label'].iloc[i] != data['label'].iloc[i - 1]:
            split_point = (data['attribute'].iloc[i] + data['attribute'].iloc[i - 1]) / 2.0
            splits.append(split_point)

    return list(set(splits))


#### e) compararea rezultatelor functiei care implementeaza ID3 pe toate atributele setului de date vs sklearn

Inaltime cu sklearn: 6 noduri, radacina fiind flipper_length_mm <= 206.50, aceasta nu este inaltimea maxima

Inaltimea propriei implementari: 7 (inaltimea maxima), radacina fiind bill_length_mm

Comparat in cazul setului de date doar cu atribute discrete, putem concluziona ca algoritmul din libraria sklearn este mai eficient pe un set de date preponderent cu valori numerice sau continue.

##### functie care calculeaza cel mai bun split din al atributului continuu

In [13]:
def find_best_split_point(data, continuous_attribute, target_attribute):

    unique_values = sorted(set(data[continuous_attribute]))
    best_split_point = None
    max_info_gain = float('-inf')

    for value in get_splits(data[continuous_attribute], data[target_attribute]):
        subset1 = data[data[continuous_attribute] <= value]
        subset2 = data[data[continuous_attribute] > value]

        info_gain = calculate_information_gain(data, target_attribute, continuous_attribute)

        if info_gain > max_info_gain:
            max_info_gain = info_gain
            best_split_point = value

    return best_split_point

print(find_best_split_point(data, "body_mass_g", target_attribute)) ##pls wait

3200.0


#### e') functia ID3 pe intreg setul de date

In [14]:
def ID3(data, target_attribute, attributes, parent_node_class=None):
   
    unique_values_count = len(data[target_attribute].unique())
    if unique_values_count == 1:
        leaf_value = data[target_attribute].iloc[0]
        return {target_attribute: leaf_value}

    if len(attributes) == 0:
        majority_class = data[target_attribute].mode().iloc[0]
        return {target_attribute: majority_class}

    best_attribute, max_info_gain = find_root_node(data, target_attribute, attributes)

    attributes = [attr for attr in attributes if attr != best_attribute]

    if best_attribute in continuous_attributes:

        split_point = find_best_split_point(data, best_attribute, target_attribute)
        data_below = data[data[best_attribute] <= split_point]
        data_above = data[data[best_attribute] > split_point]
        count_below = len(data_below)
        count_above = len(data_above)

        tree_node = {
            "node_attribute": best_attribute,
            "n_observations": {"<= " + str(split_point): count_below, "> " + str(split_point): count_above},
            "information_gain": max_info_gain,
            "values": {}
        }

        subtree_below = ID3(data_below, target_attribute, attributes,
                                  parent_node_class=f"{best_attribute} <= {split_point}")
        subtree_above = ID3(data_above, target_attribute, attributes,
                                  parent_node_class=f"{best_attribute} > {split_point}")

        tree_node["values"]["<= " + str(split_point)] = subtree_below
        tree_node["values"]["> " + str(split_point)] = subtree_above

    else:
        tree_node = {
            "node_attribute": best_attribute,
            "n_observations": data[best_attribute].value_counts().to_dict(),
            "information_gain": max_info_gain,
            "values": {}
        }

        for value in data[best_attribute].unique():
            subset_data = data[data[best_attribute] == value].drop(columns=[best_attribute])
            subtree = ID3(subset_data, target_attribute, attributes, parent_node_class=value)
            tree_node["values"][value] = subtree

    return tree_node

mytree = ID3(data,"species",all_attributes)
print(json.dumps(mytree, indent=4))

{
    "node_attribute": "bill_length_mm",
    "n_observations": {
        "<= 45.7": 193,
        "> 45.7": 140
    },
    "information_gain": 1.1582887738274013,
    "values": {
        "<= 45.7": {
            "node_attribute": "bill_depth_mm",
            "n_observations": {
                "<= 18.7": 143,
                "> 18.7": 50
            },
            "information_gain": 0.8473190472462454,
            "values": {
                "<= 18.7": {
                    "node_attribute": "flipper_length_mm",
                    "n_observations": {
                        "<= 193.0": 84,
                        "> 193.0": 59
                    },
                    "information_gain": 0.8938291494694094,
                    "values": {
                        "<= 193.0": {
                            "node_attribute": "body_mass_g",
                            "n_observations": {
                                "<= 3200.0": 26,
                                "> 3200.0": 58
     

#### e'') functia ID3 implememntata cu **sklearn**

In [15]:
def ID3_sklearn(data, target_attribute, continuous_attributes):

    X = data.drop(target_attribute, axis=1)
    y = data[target_attribute]

    # convertim valorile numerice
    for attr in continuous_attributes:
        if attr in X.columns:
            X[attr] = pd.to_numeric(X[attr], errors='coerce')

    categorical_features = X.select_dtypes(include=['object']).columns.tolist()
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(), categorical_features)
        ],
        remainder='passthrough'
    )

    X_encoded = preprocessor.fit_transform(X)
    feature_names = preprocessor.get_feature_names_out(X.columns)
    tree = DecisionTreeClassifier(criterion='entropy', random_state=42).fit(X_encoded,y)
    tree_rules = export_text(tree, feature_names=list(feature_names))
    print(tree_rules)
    
ID3_sklearn(data,"species",continuous_attributes)    

|--- remainder__flipper_length_mm <= 206.50
|   |--- remainder__bill_length_mm <= 43.35
|   |   |--- remainder__bill_length_mm <= 42.35
|   |   |   |--- remainder__bill_depth_mm <= 16.65
|   |   |   |   |--- remainder__bill_length_mm <= 39.50
|   |   |   |   |   |--- class: Adelie
|   |   |   |   |--- remainder__bill_length_mm >  39.50
|   |   |   |   |   |--- class: Chinstrap
|   |   |   |--- remainder__bill_depth_mm >  16.65
|   |   |   |   |--- class: Adelie
|   |   |--- remainder__bill_length_mm >  42.35
|   |   |   |--- remainder__flipper_length_mm <= 189.50
|   |   |   |   |--- class: Chinstrap
|   |   |   |--- remainder__flipper_length_mm >  189.50
|   |   |   |   |--- class: Adelie
|   |--- remainder__bill_length_mm >  43.35
|   |   |--- cat__island_Dream <= 0.50
|   |   |   |--- cat__sex_MALE <= 0.50
|   |   |   |   |--- class: Gentoo
|   |   |   |--- cat__sex_MALE >  0.50
|   |   |   |   |--- class: Adelie
|   |   |--- cat__island_Dream >  0.50
|   |   |   |--- remainder__bil

#### f') functia ID3 pe intreg setul de date care accepta prunnig (pe adancime)

In [16]:
def ID3_pruned(data, target_attribute, attributes, max_depth, current_depth=0, parent_node_class=None):
    
    unique_values_count = len(data[target_attribute].unique())
    if unique_values_count == 1:
        leaf_value = data[target_attribute].iloc[0]
        return {target_attribute: leaf_value}

    if len(attributes) == 0 or current_depth == max_depth:
        majority_class = data[target_attribute].mode().iloc[0]
        return {target_attribute: majority_class}

    best_attribute, max_info_gain = find_root_node(data, target_attribute, attributes)
    attributes = [attr for attr in attributes if attr != best_attribute]

    if best_attribute in continuous_attributes:
        split_point = find_best_split_point(data, best_attribute, target_attribute)
        data_below = data[data[best_attribute] <= split_point]
        data_above = data[data[best_attribute] > split_point]
        count_below = len(data_below)
        count_above = len(data_above)

        tree_node = {
            "node_attribute": best_attribute,
            "n_observations": {"<= " + str(split_point): count_below, "> " + str(split_point): count_above},
            "information_gain": max_info_gain,
            "values": {}
        }

        subtree_below = ID3_pruned(data_below, target_attribute, attributes, max_depth, current_depth + 1,
                                   parent_node_class=f"{best_attribute} <= {split_point}")
        subtree_above = ID3_pruned(data_above, target_attribute, attributes, max_depth, current_depth + 1,
                                   parent_node_class=f"{best_attribute} > {split_point}")

        tree_node["values"]["<= " + str(split_point)] = subtree_below
        tree_node["values"]["> " + str(split_point)] = subtree_above

    else:
        tree_node = {
            "node_attribute": best_attribute,
            "n_observations": data[best_attribute].value_counts().to_dict(),
            "information_gain": max_info_gain,
            "values": {}
        }

        for value in data[best_attribute].unique():
            subset_data = data[data[best_attribute] == value].drop(columns=[best_attribute])
            subtree = ID3_pruned(subset_data, target_attribute, attributes, max_depth, current_depth + 1,
                                 parent_node_class=value)
            tree_node["values"][value] = subtree

    return tree_node

mytree = ID3_pruned(data,"species",all_attributes,2)
print(json.dumps(mytree, indent=4))

{
    "node_attribute": "bill_length_mm",
    "n_observations": {
        "<= 45.7": 193,
        "> 45.7": 140
    },
    "information_gain": 1.1582887738274013,
    "values": {
        "<= 45.7": {
            "node_attribute": "bill_depth_mm",
            "n_observations": {
                "<= 18.7": 143,
                "> 18.7": 50
            },
            "information_gain": 0.8473190472462454,
            "values": {
                "<= 18.7": {
                    "species": "Adelie"
                },
                "> 18.7": {
                    "species": "Adelie"
                }
            }
        },
        "> 45.7": {
            "node_attribute": "island",
            "n_observations": {
                "Biscoe": 83,
                "Dream": 55,
                "Torgersen": 2
            },
            "information_gain": 1.064261180904602,
            "values": {
                "Torgersen": {
                    "species": "Adelie"
                },
        